In [19]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [20]:
# Define the model
EMB_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_sm = model.add_parameters((ntags, EMB_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [33]:
# A function to calculate scores for one value
def calc_scores(words):
    dy.renew_cg()
    # (EMB_SIZE,)
    cbow = dy.esum([dy.lookup(W_emb, x) for x in words])
    # ntags * EMB_SIZE
    W_sm_exp = dy.parameter(W_sm)
    # (ntags,)
    b_sm_exp = dy.parameter(b_sm)
    # score + b
    return W_sm_exp * cbow + b_sm_exp

In [36]:
for ITER in range(100):
    
    # Perform training
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    
    for words, tag in train:
        my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += my_loss.value()
        my_loss.backward()
        trainer.update()
    if ITER % 10 == 0:
        print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
    
    # Perform testing
    test_correct = 0.0
    for words, tag in dev:
        scores = calc_scores(words).npvalue()
        predict = np.argmax(scores)
    if predict == tag:
        test_correct += 1
    if ITER % 10 == 0:
        print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.5242, time=0.58s
iter 0: test acc=0.0000
iter 10: train loss/sent=0.0822, time=0.57s
iter 10: test acc=0.0005
iter 20: train loss/sent=0.0139, time=0.46s
iter 20: test acc=0.0000
iter 30: train loss/sent=0.0061, time=0.58s
iter 30: test acc=0.0005
iter 40: train loss/sent=0.0055, time=0.53s
iter 40: test acc=0.0005
iter 50: train loss/sent=0.0042, time=0.49s
iter 50: test acc=0.0000
iter 60: train loss/sent=0.0072, time=0.52s
iter 60: test acc=0.0000
iter 70: train loss/sent=0.0066, time=0.48s
iter 70: test acc=0.0005
iter 80: train loss/sent=0.0028, time=0.52s
iter 80: test acc=0.0000
iter 90: train loss/sent=0.0032, time=0.52s
iter 90: test acc=0.0005
